## Analysis and prediction of the price of the appartments in Switzerland (Extraction of the data from Immoscout24.ch)

We first extract the rental listing data from immoscout24.ch and then parse it; convert the text data to pandas data frames (thus text mining and feature extraction will be done), and finally a model to predict the appartment rental price given the features for the new listings. In this code, we only save the data for once (the very first download), then with the ImmoscoutHtml_to_csv.py script we add to this data every day we want to update our data base (rental_all.csv). 

### Save the rental data

In [1]:
import html2text
import requests
import time
import os
import datetime
from SingleAptListing_to_table import SingleAptListing_to_table
import pandas as pd

In [2]:
first_page = requests.get('https://www.immoscout24.ch/en/flat/rent/city-lausanne')
h = html2text
c = h.html2text(first_page.text)

Find the max page number:

In [3]:
def find_max_page_num(c):
    string = "Page** 1 **of** "
    l_string = len(string)
    a = c.find(string)
    b = c.find("**", a + l_string)
    return int(c[a + l_string : b-1])

In [4]:
max_page_num = find_max_page_num(c)

Save all the apartment rental listings today in the desired city:

In [ ]:
city_path = '../../rental_list/lausanne/'

In [5]:
date_today = datetime.datetime.now().strftime ("%Y%m%d")
folder_name = city_path + date_today
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
main_url = 'https://www.immoscout24.ch/en/flat/rent/city-lausanne?ci&pn='
for i in range(1, max_page_num + 1):
    page_url = main_url + str(i)
    page = requests.get(page_url)
    c = h.html2text(page.text)
    text_file_name = folder_name + '/' + 'page' + '_' + str(i) + '.txt'
    text_file = open(text_file_name, "w")
    text_file.write(c)
    text_file.close()
    time.sleep(10)    

In [9]:
folder_name += '/'
df = pd.DataFrame()
for i in range(1, max_page_num + 1):
    text_file = open(folder_name + 'page_' + str(i) + '.txt')
    c = text_file.read()
    end_c_n = c.find("Page** ")
    c = c[0:end_c_n]
    splt_listing = c.split("### [")
    for j in range(1, len(splt_listing)):
        single_apt_listing = splt_listing[j]
        tmdf = SingleAptListing_to_table(single_apt_listing)
        df = pd.concat(objs=[df, tmdf], axis = 0).reset_index(drop = True)

In [10]:
df.to_csv(folder_name + 'rental_csv.csv', index = False)

In [6]:
all_rental_data_path = city_path + 'all_rental_data'

In [15]:
if not os.path.exists(all_rental_data_path):
    os.makedirs(all_rental_data_path)
if not os.path.exists(all_rental_data_path + '/combined'):
    os.makedirs(all_rental_data_path + '/combined')

In [16]:
df.to_csv(all_rental_data_path + '/combined/rental_all.csv', index = False)